In [1]:
from copy import deepcopy

# Part 1

In [2]:
test = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

In [3]:
shapes = [ [[2,0],[3,0],[4,0],[5,0]],
           [[3,0],[2,1],[3,1],[4,1],[3,2]],
           [[2,0],[3,0],[4,0],[4,1],[4,2]],
           [[2,0],[2,1],[2,2],[2,3]],
           [[2,0],[3,0],[2,1],[3,1]] ]

def drop(shape, graph):
    flag = False
    for i in range(0, len(shape)):
        shape[i][1] -= 1
        if shape[i][1] < 1 or tuple(shape[i]) in graph.keys():
            flag = True
    if flag:
        for i in range(0, len(shape)):
            shape[i][1] += 1
            graph[tuple(shape[i])] = shape[i][1]
    
    return shape, flag, graph

def slide(shape, lr, graph):
    if lr == '<':
        dx = 1
    elif lr == '>':
        dx = -1   
    
    flag = False
    for i in range(0, len(shape)):
        shape[i][0] -= dx
        if shape[i][0] < 0 or shape[i][0] > 6 or tuple(shape[i]) in graph.keys():
            flag = True
    if flag:
        for i in range(0, len(shape)):
            shape[i][0] += dx
            
    return shape

def drop_rocks(shapes, jet, num):
    graph = {}
    graph[(0,0)] = 0
    
    ji = 0
    for i in range(0, num):       
        shape = deepcopy(shapes[i%len(shapes)])
        height = max(graph.values())
        
        #spawn rock
        for i in range(0, len(shape)):
            shape[i][1] += height+4
            
        flag = False
        while flag == False:
            j = jet[ji%len(jet)]
            ji += 1
            shape = slide(shape, j, graph)
            shape, flag, graph = drop(shape, graph)
    return max(graph.values())

In [4]:
drop_rocks(shapes, test, 2022)

3068

In [5]:
with open('day17_input.txt', 'r') as f:
    inpt = f.readlines()
    f.close()
inpt = inpt[0].strip()
print('Part 1 result:', drop_rocks(shapes, inpt, 2022))

Part 1 result: 3149


# Part 2

In [6]:
def drop_rocks_part2(shapes, jet):
    graph = {}
    graph[(0,0)] = 0
    history = {}
    first = True
    
    ji = 0
    for i in range(0, 1000000000000):       
        shape = deepcopy(shapes[i%len(shapes)])
        height = max(graph.values())
        
        #let cycle settle, so ignore (burn) first loop over jet
        if i > len(jet):
            if (i%len(shapes),ji%len(jet)) not in history.keys():
                history[(i%len(shapes),ji%len(jet))] = [height, i]
            elif first and i>5:
                first_history = [height, i]
                delta_height = height - history[(i%len(shapes),ji%len(jet))][0]
                delta_rock = i - history[(i%len(shapes),ji%len(jet))][1]
                first = False
            else:
                return delta_height, delta_rock, history, (i%len(shapes),ji%len(jet))
        
        #spawn rock
        for i in range(0, len(shape)):
            shape[i][1] += height+4
            
        flag = False
        while flag == False:
            j = jet[ji%len(jet)]
            ji += 1
            shape = slide(shape, j, graph)
            shape, flag, graph = drop(shape, graph)
            
    return max(graph.values())

def run_part2(shapes, jet):
    #find when cycle repeats
    delta_height, delta_rock, history, key = drop_rocks_part2(shapes, jet)
    #get burnin period
    burn_height = history[key][0]
    burn_rocks = history[key][1]
    
    #find number of cycles after burnin peroid and it's height
    rocks = 1000000000000-burn_rocks
    cycles = int(rocks//delta_rock)
    height = cycles*delta_height
    
    #find missing height. The remainder will appear right after the burnin period
    remainder = int(rocks%delta_rock)
    xs_height = drop_rocks(shapes, jet, burn_rocks+remainder)
    height += xs_height
    
    print('Part 2 result:', height)
    
run_part2(shapes, test)

Part 2 result: 1514285714288


In [7]:
run_part2(shapes, inpt)

Part 2 result: 1553982300884
